In [1]:
import torch
import torch.nn.functional as F
print(torch.__version__)

# The PyG built-in GCNConv
from torch_geometric.nn import GCNConv

import torch_geometric.transforms as T
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

1.10.2


In [2]:
#dataset
dataset_name = 'ogbn-arxiv'
dataset = PygNodePropPredDataset(name=dataset_name,
                                 transform=T.ToSparseTensor())

data = dataset[0]
device = 'cuda' if torch.cuda.is_available() else 'cpu'
data=data.to(device)

data.adj_t=data.adj_t.to_symmetric()
split_idx=dataset.get_idx_split()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
train_idx=split_idx['train'].to(device)


In [3]:

data.adj_t=data.adj_t.to_symmetric()
split_idx=dataset.get_idx_split()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
train_idx=split_idx['train'].to(device)


Model
![picture](cs224w-colab2-3.png)

In [4]:
#models <-Construct the network as showing in the figure
class GCN(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim,output_dim,num_layers,dropout,return_embeds=False):
    super(GCN, self).__init__()
    self.convs=None
    self.bns=None
    self.dropout=dropout
    self.num_layers=num_layers
    self.return_embeds=return_embeds
    self.convs=torch.nn.ModuleList()
    self.convs.append(GCNConv(input_dim, hidden_dim))
    for i in range(num_layers-2):
      self.convs.append(GCNConv(hidden_dim,hidden_dim))
    self.convs.append(GCNConv(hidden_dim,output_dim))
    self.bns=torch.nn.ModuleList([torch.nn.BatchNorm1d(hidden_dim) for i in range(num_layers-1)])
    self.softmax=torch.nn.LogSoftmax()
  
  def reset_parameters(self):
    for conv in self.convs:
      conv.reset_parameters()
    for bn in self.bns:
      bn.reset_parameters()
  def forward(self,x, adj_t):
    for i in range(self.num_layers-1):
      x=self.convs[i](x, adj_t)
      x=self.bns[i](x)
      x=F.relu(x)
      x=F.dropout(x,self.dropout,self.training)
    x=self.convs[-1](x,adj_t)
    if not self.return_embeds:
      x=self.softmax(x)
    return x


In [5]:
print(data.x.shape)
#取多行
print(data.x[train_idx].shape)
#取一行 降维
print(data.x[0].shape)
#取一列 降维
print(data.x[:,0].shape)
print(data.x[train_idx,0].shape)


torch.Size([169343, 128])
torch.Size([90941, 128])
torch.Size([128])
torch.Size([169343])
torch.Size([90941])


Train

In [6]:
def train(model, data, train_idx, optimizer, loss_fn):
    # TODO: Implement this function that trains the model by 
    # using the given optimizer and loss_fn.
    model.train()
    loss = 0

    ############# Your code here ############
    ## Note:
    ## 1. Zero grad the optimizer
    ## 2. Feed the data into the model
    ## 3. Slicing the model output and label by train_idx
    ## 4. Feed the sliced output and label to loss_fn
    ## (~4 lines of code)
    optimizer.zero_grad()
    #x=data.x[train_idx]
    y=data.y[train_idx,0]
    #why  model(data.x[train_idx]) is wrong ?
    l=loss_fn(model(data.x,data.adj_t)[train_idx],y)
    l.backward()
    optimizer.step()
    return l.item()

In [7]:
# Test function here
@torch.no_grad()
def test(model, data, split_idx, evaluator):
    # TODO: Implement this function that tests the model by 
    # using the given split_idx and evaluator.
    model.eval()
    #Sets the module in evaluation mode.

    # The output of model on all data
    out = None

    ############# Your code here ############
    ## (~1 line of code)
    ## Note:
    ## 1. No index slicing here
    out=model(data.x,data.adj_t)
    #########################################
    y_pred=torch.argmax(out,dim=-1,keepdim=True)
    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']
    return train_acc, valid_acc, test_acc
  

In [8]:
#hyperparameters
args = {
    'device': device,
    'num_layers': 3,
    'hidden_dim': 256,
    'dropout': 0.5,
    'lr': 0.01,
    'epochs': 100,
}
#instanize
model = GCN(data.num_features, args['hidden_dim'],
            dataset.num_classes, args['num_layers'],
            args['dropout']).to(device)
evaluator = Evaluator(name='ogbn-arxiv')
args

{'device': 'cuda',
 'num_layers': 3,
 'hidden_dim': 256,
 'dropout': 0.5,
 'lr': 0.01,
 'epochs': 100}

In [9]:
#debug module is
#model(data.x,data.adj_t)

In [10]:
optimizer=torch.optim.Adam(model.parameters(),args['lr'])
loss_fn=F.nll_loss
#train(model,data,train_idx, optimizer, loss_fn)

Training

In [11]:
#train
import copy
# reset the parameters to initial random value
model.reset_parameters()
optimizer=torch.optim.Adam(model.parameters(),args['lr'])
loss_fn=F.nll_loss

best_acc=0
best_model=None
# skip node-level
args['epochs']=1
for epoch in range(args['epochs']):
  loss=train(model,data,train_idx, optimizer, loss_fn)
  train_acc, valid_acc, test_acc=test(model,data,split_idx,evaluator)
  if valid_acc>best_acc:
    best_acc=valid_acc
    best_model=copy.deepcopy(model)
  print(f'epoch:{epoch:02d},',f'train_acc:{train_acc:.2f},',f'valid_acc:{valid_acc:.2f}',f'test_acc:{test_acc:.2f}')

epoch:00, train_acc:0.17, valid_acc:0.25 test_acc:0.23


C:\Users\qwe\AppData\Local\Temp\ipykernel_15204\1815774667.py:31: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x=self.softmax(x)


# graph level


dataprocess&hyperparam

In [12]:
from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
from torch_geometric.data import DataLoader
from tqdm.notebook import tqdm

dataset=PygGraphPropPredDataset(name='ogbg-molhiv')
device='cuda' if torch.cuda.is_available() else 'cpu'
split_idx=dataset.get_idx_split()

print(dataset.task_type)
print(dataset.data.y)

binary classification
tensor([[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]])


In [13]:
train_loader=DataLoader(dataset[split_idx["train"]],batch_size=32,shuffle=True,num_workers=0)
valid_loader=DataLoader(dataset[split_idx["valid"]], batch_size=32,shuffle=False,num_workers=0)
test_loader=DataLoader(dataset[split_idx["test"]], batch_size=32,shuffle=False,num_workers=0)
args = {
    'device': device,
    'num_layers': 5,
    'hidden_dim': 256,
    'dropout': 0.5,
    'lr': 0.001,
    'epochs': 30,
}


C:\Users\qwe\miniconda3\envs\gnn\lib\site-packages\torch_geometric\deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Model that reusing GCN

In [14]:
from ogb.graphproppred.mol_encoder import AtomEncoder
from torch_geometric.nn import global_add_pool, global_mean_pool

class GCN_Graph(torch.nn.Module):
    def __init__(self,hidden_dim,output_dim,num_layers,dropout):
        super(GCN_Graph,self).__init__()
        self.node_encoder=AtomEncoder(hidden_dim)
        self.gnn_node=GCN(hidden_dim,hidden_dim,hidden_dim,num_layers,dropout,return_embeds=True)
        self.pool=global_mean_pool
        self.linear=torch.nn.Linear(hidden_dim,output_dim)
    def reset_parameters(self):
        self.gnn_node.reset_parameters()
        self.linear.reset_parameters()
    
    def forward(self,batched_data):
        x,edge_index,batch=batched_data.x,batched_data.edge_index, batched_data.batch
        x=self.node_encoder(x)
        x=self.gnn_node(x,edge_index)
        x=self.pool(x,batch)
        out=self.linear(x)

        return out

       

train
for batch 

1. to device
2. zero grad
3. loss(y,y_hat)
4. backward
5. step

return loss.toitem

## eval
for batch_size
batch.to(device)
get list y,y_hat
eval({"y":y,"y_hat":y_hat}})

In [19]:
def train(model,device,data_loader,optimizer,loss_fn):
    model.train()
    loss=0
    for step,batch in enumerate(tqdm(data_loader,desc="Iteration on single batch")):
        batch=batch.to(device)
        optimizer.zero_grad()
        out=model(batch).view(-1)
        train_label=batch.y.view(-1)
        loss=loss_fn(out.float(),train_label.float())
        loss.backward()
        optimizer.step()
    return loss.item()

def eval(model,device,data_loader,evaluator):
    model.eval()
    y_true=[]
    y_pred=[]
    #batch.y.shape?
    for step,batch in enumerate(tqdm(data_loader,desc="Iteration on eval")):
        batch=batch.to(device)
        with torch.no_grad():
            t=model(batch)
        #bug 写反了
        y_pred.append(t.detach().cpu())
        y_true.append(batch.y.view(t.shape).detach().cpu())
    #y_true.shape?
    y_true=torch.cat(y_true,dim=0).numpy()
    y_pred=torch.cat(y_pred,dim=0).numpy()
    #y_true[0][0]=1
    input_dict={"y_true":y_true,"y_pred":y_pred}
    return evaluator.eval(input_dict)

## training
for epoch
 loss

 get acc
 best model

In [21]:
import copy 
model=GCN_Graph(args['hidden_dim'],dataset.num_tasks,args['num_layers'],args['dropout']).to(device)
evaluator=Evaluator(name='ogbg-molhiv')
model.reset_parameters()
optimizer=torch.optim.Adam(model.parameters(),lr=args['lr'])
loss_fn=torch.nn.BCEWithLogitsLoss()

best_model=None
best_valid_acc=0

for epoch in range(1,1+args['epochs']):
    print("training...")
    loss=train(model,device,train_loader,optimizer,loss_fn)
    print('Eval...')
    val_result=eval(model,device,valid_loader,evaluator)
    valid_acc=val_result[dataset.eval_metric]
    if valid_acc>best_valid_acc:
        best_valid_acc=valid_acc
        best_model=model
    print(f'Valid:{100*valid_acc:.2f}%')
    

training...


Iteration on single batch:   0%|          | 0/1029 [00:00<?, ?it/s]

Eval...


Iteration on eval:   0%|          | 0/129 [00:00<?, ?it/s]

Valid:72.33%
training...


...

Valid:77.38%
training...


Iteration on single batch:   0%|          | 0/1029 [00:00<?, ?it/s]

Eval...


Iteration on eval:   0%|          | 0/129 [00:00<?, ?it/s]

Valid:76.93%


In [22]:
print(best_valid_acc)

0.7947163433274544
